In [1]:
#!pip --version

In [1]:
!pip install --no-cache-dir --upgrade --force avitech --no-index --find-links  C:\GIT\avitech\package\dist

Looking in links: c:\GIT\avitech\package\dist
Processing c:\git\avitech\package\dist\avitech-0.1.5.tar.gz
  Created wheel for avitech: filename=avitech-0.1.5-py3-none-any.whl size=89999 sha256=67709da30d3d69165932623b5068832a4383a7e180a506fbab503f9c08fa94a3
  Stored in directory: C:\Users\Артем\AppData\Local\Temp\pip-ephem-wheel-cache-na5xwa3n\wheels\cf\93\ec\bc4720e2c424da239ec4f7d3d9cc2eb5463ac228c6ac7cb35b
Successfully built avitech
  Attempting uninstall: avitech
    Found existing installation: avitech 0.1.5
    Uninstalling avitech-0.1.5:
      Successfully uninstalled avitech-0.1.5


In [8]:
import logging
logging.basicConfig(filename='C:\\Git\\avitech\\package\\test.log', level=logging.DEBUG) #encoding='utf-8', 

In [1]:
from avi.mine.enums import *
#from avi.mine.client import Client
from avi.mine import map
"""
game = Player(server_name) 
game.play() # отобразить игру. статус игрока меняется на 1
while game.active():  # active проверяет статус сервера и игрока. если сервер выключился - выходит, если игрок умер (статус 2, выполняем game.reconnect()
    objs = game.get_objs() # select from map
    game.move('right')
"""


class Player():
    def __init__(self, server_name, ava = avatar.cowboy):
        self.client = Client()
        self.client.connect(server_name, ava)

    def active(self):
        # проверить статус сервера и игрока
        self.client.refresh_user()
        self.client.refresh_server()
        return self.client.check_state()

    def play(self, autostop = False):
        # запустить игру
        self.client.start(autostop)

    def move_right(self):
        # идти вправо
        return self.client.make_action(action.move_right)

    def move_left(self):
        # идти влево
        return self.client.make_action(action.move_left) 

    def move_up(self):
        # идти вверх
        return self.client.make_action(action.move_up) 

    def move_down(self):
        # идти вниз
        return self.client.make_action(action.move_down) 

    def move(self, dir):
        # двигаться. dir: right, left, down, up
        if dir == 'up':
            return self.move_up()
        elif dir == 'down':
            return self.move_down()
        elif dir == 'left':
            return self.move_left()
        elif dir == 'right':
            return self.move_right()
        return False

    def pick(self):
        # взять сокровище
        return self.client.make_action(action.pick) 

    def hide(self):
        # спрятаться
        return self.client.make_action(action.hide) 

    def get_pos(self):
        # получить координаты игрока (row,col)
        self.client.refresh_user()
        return (self.client.user['row'], self.client.user['col'])

    def get_objs(self):
        # осмотреться
        self.client.refresh_user()
        return map.get_objs(self.client.server, self.client.user['row'], self.client.user['col'], con=self.client.con)

    def get_chests(self):
        # получить список координат сокровищ [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.chest, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests

    def get_guards(self):
        # получить список координат стражей [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.guard, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests        

    def get_players(self):
        # получить список координат игроков [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.player, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests                

    def get_walls(self):
        # получить список координат препятствий [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.wall, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests                        

    def get_dir(self, pos_goal):
        """ Функция определения направления движения игрока
        Параметры:
        pos_user - кортеж с координатой игрока
        pos_goal - кортеж с координатой цели
        Возвращаемое значение:
        dir - строка с кодом направления "up", "down", "left", "right"
        """
        pos_user = (self.client.user['row'], self.client.user['col'])
        delta_row = pos_goal[0] - pos_user[0]    # вычислить разницу строк
        delta_col = pos_goal[1] - pos_user[1]    # вычислить разницу столбцов
        dir = ""
        
        if abs(delta_row) > abs(delta_col):  # если по вертикали идти больше, чем по горизонтали - идем вверх или вниз
            if delta_row < 0:
                dir = "up"
            elif delta_row > 0:
                dir = "down"
        else:
            if delta_col < 0:
                dir = "left"
            elif delta_col > 0:
                dir = "right"
                
        return dir    
    
    def get_path(self, pos_goal = (0,0)):
        """найти кратчайший путь с учетом препятствий
        params:
        pos_goal - координаты цели
        return:
        список ячеек, по которым следует идти
        """
        return map.find_path(self.client.server, (self.client.user['row'], self.client.user['col']), pos_goal, con=self.client.con)

    def get_nearest(self, objects):
        "Найти ближайший объект из списка"
        import numpy
        import builtins
        pos = (self.client.user['row'], self.client.user['col'])
        delta_pos = list(builtins.map(lambda x: tuple(numpy.subtract(x, pos)), objects))
        deltas = list(builtins.map(lambda x:abs(x[0]) + abs(x[1]), delta_pos))
        return objects[deltas.index(min(deltas))]

In [3]:
path = str(pathlib.Path().absolute()) + "\\avi\\mine"

NameError: name 'pathlib' is not defined

In [9]:
import getpass
from time import sleep
from threading import Thread
import pathlib
from datetime import datetime

from ipywidgets import widgets, Label, HTML, HBox, Image, VBox, Box, HBox, Layout, Button, GridBox
from ipycanvas import Canvas, MultiCanvas, hold_canvas
from IPython.display import display, clear_output

from avi.mine.enums import *
from avi.mine.server import Server
from avi.mine.data import server
from avi.mine.data.user import *
from avi.mine.data import event
from avi.mine import map
from avi.mine.data import base

"""
Класс client как перегрузка DLGameTask в котором перегружаем работу с картой.
Вместо списка levelmap обращаемся к базе
Вместо выполнения действия пишем в очередь и ждем
Остальное все сохраняется
+ добавляем координаты сокровищ (зажигать на карте?)
+ добавляем справа список пользователей со счетом
+ статус подключения. кнопка стоп. кнопка реконнект
+ если умер, то киллс + 1 и на воскрешение
"""
REDRAW_LAG = 0.2
KILL_LAG = 2
CELL_PIXELS = 40

#path = str(pathlib.Path(__file__).parent.absolute())
path = str(pathlib.Path().absolute()) + "\\avi\\mine"

class Client():
    def __init__(self):
        self.user_name = getpass.getuser()
        self.drawer = None

    def connect(self, server_name, ava = avatar.cowboy):
        self.con = base.connect()
        self.server = server.find_server(server_name, self.con)
        if self.server is None:
            raise ValueError('Сервер {} не найден'.format(server_name))
        self.user = find_or_create_user(self.server['id'], self.user_name, ava, self.con)
        self.userid = self.user['id']

        if self.send_event(action.spawn) == action_state.processed:
            print('Ожидание подключения к серверу {}...'.format(server_name))
            self.user = find_user(self.userid)
            self.user['state'] = player_state.active
            self.user['score'] = 0
            self.user['kills'] = 0
            update_user(self.user, self.con)
            clear_output(wait=True)
            #print('Пользователь {} успешно подключен к серверу {}...'.format(self.user_name, server_name))
        else:
            raise ValueError('Не удалось подключить пользователя {}'.format(self.user_name))

    def refresh_user(self):
        self.user = find_user(self.userid, self.con)

    def refresh_server(self):
        self.server = server.find_server_id(self.server['id'], self.con)

    def check_state(self):
        return self.user['state'] != player_state.inactive and self.server['state'] == server_state.active

    def send_event(self, act=action.spawn):
        self.refresh_user()
        if self.user['state'] in [player_state.inactive, player_state.killed]  and act != action.spawn:
            return action_state.rejected

        return event.send_event(self.server['id'], self.userid, act, self.con)

    def make_action(self, act = action.spawn):
        state = self.send_event(act)
        #self.redraw()
        return state == action_state.processed

    def start(self, autostop = False):
        self.autostop = autostop
        if self.drawer != None:
            self.stop()    
        self.drawer = Drawer(self)
        self.drawer.display()
        self.drawer.draw_base()
        self.drawer.start()
        return True

    def stop(self, b = None):
        logging.error('Stop')
        self.drawer.stop = True
        if self.autostop:
            raise Exception('Game over')

########################################################################################################################
class Drawer(Thread):
    def __init__(self, client):
        self.client = client
        self.stop = False
        self.map = None
        self.users = None
        self.user_scores = None
        self.init_images()
        super(Drawer, self).__init__()

    def init_images(self):

        self.images_ava = {}
        self.images_ava_hide = {}        
        avatars = [avatar.cowboy, avatar.stan, avatar.rock, avatar.pig, avatar.glass, avatar.dipper, avatar.zoose, \
                    avatar.super, avatar.garry, avatar.chui, avatar.lord, avatar.bill,]

        for ava in avatars:
            self.images_ava[(obj.player, ava)] = Image.from_file(path + '/images/avatar/{}.png'.format(ava))
            self.images_ava_hide[ava] = Image.from_file(path + '/images/avatar/{}_h.png'.format(ava))            

        for ava in ava_guard.items():
            self.images_ava[(obj.guard, ava)] = Image.from_file(path + '/images/{}.png'.format(ava))

        for ava in ava_chest.items():
            self.images_ava[(obj.chest, ava)] = Image.from_file(path + '/images/{}.png'.format(ava))
        self.images_ava[(obj.wall, '')] = Image.from_file(path + '/images/wall.jpg')
        self.image_killed = Image.from_file(path + '/images/player_loss.png')
        self.image_space = Image.from_file(path + '/images/space.jpg')

    def create_panel(self):
        # images button https://fontawesome.com/v4.7.0/icons/
        items_layout = Layout( width='80px')     # override the default width of the button to 'auto' to let the button grow

        box_layout = Layout(display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            border='solid')
                            #,width='30%')

        self.label_game_status = Label('Подключение к серверу...')
        
        self.panel_users = GridBox(children=[], layout=Layout(grid_template_columns="80px 80px 30px"))

        # панель запуска-остановки-паузы
        #b_play = Button(description='ИГРА',icon = 'fa-play', layout = items_layout)
        b_stop = Button(description='СТОП',icon = 'fa-stop', layout = items_layout)
        
        #b_play.on_click(self.client.start) # client.start) 
        b_stop.on_click(self.client.stop) 

        panel = VBox([HBox([Label('Сервер:'), Label(self.client.server['name']), self.label_game_status]),
                          HBox([Label('Игрок:'),Label(self.client.user_name), b_stop]),
                          self.panel_users],
                      layout=box_layout)    
        return panel          
        
    def update_panel(self, b=None):
        state_caption = 'Активен' if self.client.server['state'] == server_state.active else 'Остановлен'
        if self.label_game_status.value != state_caption:
            self.label_game_status.value = state_caption
        user_scores = {}
        user_ava = {}
        for user_rec in self.users:
            user_scores[user_rec['name']] = user_rec['score'] - user_rec['kills']*3
            user_ava[user_rec['name']] = user_rec['avatar']
        changed = user_scores.values() != self.user_scores.values() if not self.user_scores is None else True

        if changed:
            items = []
            for user_name, score in sorted(user_scores.items(), key=lambda x: x[1], reverse = True) :
                items.append(Label(user_name))
                items.append(Label(user_ava[user_name]))
                items.append(Label(str(score)))
            self.panel_users.children = items
            self.user_scores = user_scores
                
    def display(self):
        multi = MultiCanvas(2, width=self.client.server['mapsize_x'] * CELL_PIXELS, height=self.client.server['mapsize_y'] * CELL_PIXELS)
        #multi[0].fill_style = 'black'
        #multi[0].fill_rect(0, 0, multi.size[0], multi.size[1])
        self.canvas_base = multi[0]
        self.canvas = multi[1]
        
        panel = self.create_panel()
        
        self.output = widgets.Output()
        self.output.clear_output()
        #display(VBox([Image.from_file(path + '/images/header.jpg', width=200,height=40), HBox([multi])]), self.output)        
        display(HBox([multi, panel]), self.output)        

    def draw_base(self):
        self.update_map()
        with hold_canvas(self.canvas_base):
            self.canvas_base.clear()
            self.canvas_base.fill_style = 'black'
            self.canvas_base.fill_rect(0, 0, self.canvas_base.size[0], self.canvas_base.size[1])
            for row in range(len(self.map)):
                for col in range(len(self.map[row])):
                    cell = self.map[row][col]
                    if cell['obj'] == obj.wall:
                        self.canvas_base.draw_image(self.images_ava[(obj.wall, '')], col * CELL_PIXELS, row * CELL_PIXELS)
        
    def redraw(self):
        if self.update_map():
            with hold_canvas(self.canvas):
                for row in range(len(self.map)):
                    for col in range(len(self.map[row])):
                        cell = self.map[row][col]
                        if cell['obj'] == obj.player:
                            for user_i in self.users:
                                if user_i['id'] == cell['userid']:
                                    if user_i['state'] == player_state.hide:
                                        self.canvas.clear_rect(col * CELL_PIXELS, row * CELL_PIXELS, CELL_PIXELS, CELL_PIXELS)
                                        self.canvas.draw_image(self.images_ava_hide[cell['image']], col * CELL_PIXELS, row * CELL_PIXELS)
                                    elif user_i['state'] == player_state.active:
                                        self.canvas.clear_rect(col * CELL_PIXELS, row * CELL_PIXELS, CELL_PIXELS, CELL_PIXELS)
                                        self.canvas.draw_image(self.images_ava[(cell['obj'], cell['image'])], col * CELL_PIXELS, row * CELL_PIXELS)            
                                    elif user_i['state'] == player_state.killed:
                                        self.canvas.draw_image(self.image_killed, col * CELL_PIXELS, row * CELL_PIXELS)            
                                    elif user_i['state'] == player_state.inactive:
                                        self.canvas.clear_rect(col * CELL_PIXELS, row * CELL_PIXELS, CELL_PIXELS, CELL_PIXELS)
                        elif cell['obj'] in [obj.chest, obj.guard]:                                    
                            self.canvas.draw_image(self.images_ava[(cell['obj'], cell['image'])], col * CELL_PIXELS, row * CELL_PIXELS)
                        elif cell['obj'] == obj.space:                                                                
                            self.canvas.clear_rect(col * CELL_PIXELS, row * CELL_PIXELS, CELL_PIXELS, CELL_PIXELS)
            self.update_panel()
                                       
    def update_map(self):
        changed = False
        new_map = map.get_all(self.client.server, self.client.con)
        if self.map is None or self.map != new_map:
            self.map = new_map
            changed = True

        new_users = find_all_users(self.client.server['id'], self.client.con)
        if self.users is None or self.users != new_users:
            self.users = new_users
            changed = True
        self.client.refresh_user()
        self.client.refresh_server()
        return changed
    
    def run(self):
        while(True):
            try:
                if not self.client.check_state() or self.stop: 
                    self.label_game_status.value = 'Остановлен'
                    self.output.clear_output()
                    return
                self.redraw()

                if self.client.user['state'] == player_state.killed:
                    if (datetime.utcnow() - self.client.user['kill_dt']).total_seconds() >= KILL_LAG:
                        event.send_event(self.client.server['id'], self.client.userid, action.spawn, self.client.con)

                sleep(REDRAW_LAG)
            except Exception as e: 
                pass


In [10]:
game = Player('4cubes', avatar.lord) 
game.play()

while True:
    pos = game.get_pos()           # получить позицию игрока
    chests = game.get_chests()     # получить список координат сокровищ
    chest = game.get_nearest(chests) # найти самое близкое сокровище от игрока
    print(chest)
    cells = game.get_path(chest)
    print(cells)
    for cell in cells[1:]:
        dir = game.get_dir(cell)
        if dir:                     # проверить, что направление выбрано
            game.move(dir)          # выполнить движение
        if obj.chest in game.get_objs().values():   # проверить, есть ли сокровище по близости. 
            game.pick()   
#game.client.drawer = Drawer(game.client)
#game.client.drawer.display()
#game.client.drawer.draw_base()

Output()

(2, 0)
[(0, 0), (1, 0), (2, 0)]
(4, 5)
[(2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
(5, 6)
[(4, 5), (5, 5), (5, 6)]
(5, 3)
[(5, 6), (5, 5), (5, 4), (5, 3)]
(2, 4)
[(5, 3), (5, 4), (4, 4), (3, 4), (2, 4)]
(2, 9)
[(2, 4), (2, 5), (1, 5), (1, 6), (1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(2, 9)
[(1, 7), (1, 8), (2, 8), (2, 9)]
(4, 8)
[(5, 9), (5, 8), (4, 8)]
(5, 7)
[(4, 8), (4, 7), (5, 7)]
(6, 9)
[(5, 7), (5, 8), (6, 8), (6, 9)]
(9, 9)
[(6, 9), (7, 9), (8, 9), (9, 9)]
(8, 5)
[(9, 9), (9, 8), (9, 7), (9, 6), (9, 5), (8, 5)]
(8, 2)
[(8, 5), (8, 4), (8, 3), (8, 2)]
(9, 3)
[(8, 2), (9, 2), (9, 3)]
(8, 7)
[(8, 4), (8, 5), (8, 6), (8, 7)]
(7, 8)
[(8, 7), (8, 8), (7, 8)]


KeyboardInterrupt: 

In [ ]:
game.client.stop()

In [16]:
game.client.check_state()

2020-11-13 17:28:55.183348


True

2020-11-13 17:28:55.390344
2020-11-13 17:28:55.593346
2020-11-13 17:28:55.830343
2020-11-13 17:28:56.034346
2020-11-13 17:28:56.238345
2020-11-13 17:28:56.442347
2020-11-13 17:28:56.646376
2020-11-13 17:28:56.850377
2020-11-13 17:28:57.054376
2020-11-13 17:28:57.258375
2020-11-13 17:28:57.462379
2020-11-13 17:28:57.724385
2020-11-13 17:28:57.928375
2020-11-13 17:28:58.132378
2020-11-13 17:28:58.336375
2020-11-13 17:28:58.540377
2020-11-13 17:28:58.744377
2020-11-13 17:28:58.948375
2020-11-13 17:28:59.152378
2020-11-13 17:28:59.356379
2020-11-13 17:28:59.615380
2020-11-13 17:28:59.819377
2020-11-13 17:29:00.023377
2020-11-13 17:29:00.227377
2020-11-13 17:29:00.431377
2020-11-13 17:29:00.635382
2020-11-13 17:29:00.840376
2020-11-13 17:29:01.043376
2020-11-13 17:29:01.247378
2020-11-13 17:29:01.518377
2020-11-13 17:29:01.754389
2020-11-13 17:29:01.958376
2020-11-13 17:29:02.162376
2020-11-13 17:29:02.366375
2020-11-13 17:29:02.569377
2020-11-13 17:29:02.817376
2020-11-13 17:29:03.025381
2

In [12]:
game.client.drawer.update_map()

True

In [11]:
game.client.drawer.stop

False

In [7]:
#game.move_up()
#game.move_left()
#game.move_down()
game.move_right()
#for _ in range(3):
#    game.move_left()
#game.pick()

True